## Example: Use Meeting Generator to generate artificial meeting datasets

In [ ]:
#Uncomment the following two lines to enable the logging messages of the meeting generation for a more detailed insight in the generation process
#import logging
#logging.basicConfig(level=logging.INFO)

In [ ]:
import mms_msg
import lazy_dataset

In [ ]:
#Prepare source dataset from which certain statistics are sampled
from mms_msg.databases.single_speaker.librispeech.database import LibriSpeech8kHz
libri_db = LibriSpeech8kHz()
libri_ds = libri_db.get_dataset('test_clean')

source_ds = mms_msg.sampling.source_composition.get_composition_dataset(libri_ds, num_speakers=3)
source_ds = source_ds.map(mms_msg.sampling.pattern.meeting.MeetingSampler(duration=960000, # in samples
                                                            overlap_sampler=mms_msg.sampling.pattern.meeting.overlap_sampler.UniformOverlapSampler(max_concurrent_spk=2, p_silence=0.1, maximum_silence=16000, maximum_overlap=64000)
                                                           )(libri_ds))
print('Number of Meetings:', len(source_ds))

In [ ]:
import paderbox as pb
with pb.visualization.figure_context():
    mms_msg.visualization.plot.plot_mixture(source_ds[0])

In [ ]:
#Estimate statistics of source dataset
from mms_msg.sampling.pattern.meeting.state_based.dataset_statistics_estimation import MeetingStatisticsEstimatorMarkov
ds_estimator = MeetingStatisticsEstimatorMarkov(source_ds)

model = ds_estimator.model
silence_dist = ds_estimator.silence_distribution
overlap_dist = ds_estimator.overlap_distribution

In [ ]:
#Generate new Meeting using the normal mms_msg sampling pipeline
from mms_msg.sampling.pattern.meeting.state_based.meeting_generator import MeetingGeneratorMap
input_ds = libri_db.get_dataset('test_clean')

# Fitting of the generator
generator_map = MeetingGeneratorMap(source_dataset=source_ds, input_dataset=input_ds)

In [ ]:
# The generator must be mapped to a composition dataset with the same number of speakers as the source dataset
ds = mms_msg.sampling.source_composition.get_composition_dataset(input_ds, num_speakers=3)[0:1000]
output_ds1 = ds.map(generator_map)

In [ ]:
#Estimate statistics of generated dataset
ds_estimator2 = MeetingStatisticsEstimatorMarkov(output_ds1)

model2 = ds_estimator2.model
silence_dist2 = ds_estimator2.silence_distribution
overlap_dist2 = ds_estimator2.overlap_distribution

In [ ]:
#Compare statistics of the source dataset and the generated dataset
import matplotlib.pyplot as plt
from mms_msg.sampling.utils.distribution_model import statistical_distance
print(model)
print(model2)

print('Statistical distance silence:', statistical_distance(silence_dist, silence_dist2))
print('Statistical distance overlap:', statistical_distance(overlap_dist, overlap_dist2))

fig, ax = plt.subplots(1,2,figsize=(11, 4))
fig.subplots_adjust(wspace = 0.25)
silence_dist.plot(ax=ax[0])
silence_dist2.plot(ax=ax[0])
ax[0].set_title('silence')
overlap_dist.plot(ax=ax[1])
overlap_dist2.plot(ax=ax[1])
ax[1].set_title('overlap')
plt.show()

In [ ]:
#Alternative way to use the meeting generator
from mms_msg.sampling.pattern.meeting.state_based.meeting_generator import MeetingGenerator
generator = MeetingGenerator()
generator.fit(source_ds)

In [ ]:
# Here the number of speakers can differ from the source dataset (When the underlying transition model supports a variable number of speakers)
output_ds2 = generator.generate(input_dataset = input_ds, num_speakers = 3, num_meetings = 5000)

## Adapting the meeting generator to show a custom behavior during meeting generation

It is also possible to adapt the behavior of the meeting generator, this can be done by changing out the two main componenents of it:
* transition_model: Responsible for generating the seqeuence of speakers and the transition types (actions) between them.
* action_handler: Responsible for sampling a source that fits to the action and id of the next speaker, that were generated by the transition_model.

The custom classes must inherit from the following classes and implement the respective methods:
* Transition model: mms_msg.sampling.pattern.meeting.state_based.transition_model.SpeakerTransitionModel
* Action handler: mms_msg.sampling.pattern.meeting.state_based.action_handler.ActionHandler

In [ ]:
from mms_msg.sampling.pattern.meeting.state_based.transition_model import SpeakerTransitionModel
from mms_msg.sampling.pattern.meeting.state_based.action_handler import ActionHandler

class CustomTransitionModel(SpeakerTransitionModel):
    def __init__(self):
        pass
    # Implementation of class methods:
    # ...
    # --------------------------------
    
    
class CustomActionHandler(ActionHandler):
    def __init__(self):
        pass
    # Implementation of class methods:
    # ...
    # --------------------------------

In [ ]:
#from mms_msg.sampling.pattern.meeting.state_based.weighted_meeting_sampler import WeightedMeetingSampler
#meeting_sampler = WeightedMeetingSampler(transition_model=CustomTransitionModel(), action_handler=CustomActionHandler())(input_dataset)